In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Создаем SparkSeesion
spark = SparkSession.builder \
        .appName("Module1_Basics") \
        .config("spark.sql.adaptive.enabled", "true") \
        .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
        .getOrCreate()

# Проверяем версию Spark
print(f"Spark Version: {spark.version}")
print(f"Spark UI: http://localhost:4040")

Spark Version: 3.5.0
Spark UI: http://localhost:4040


#### Создание DataFrame различными способами

In [6]:
# 1. Создание из списка Python
data = [
    ("Alice", 25, "Engineer"),
    ("Bob", 30, "Manager"),
    ("Charlie", 35, "Developer"),
    ("Diana", 28, "Analyst")
]
columns = ["name", "age", "job"]

df1 = spark.createDataFrame(data, columns)
df1.show()

# 2. Создание из pandas DataFrame
import pandas as pd
pandas_df = pd.DataFrame({
    'name': ['Eve', 'Frank', 'Grace'],
    'age': [32, 27, 29],
    'job': ["Designer", "QA", "Product Manager"]
})

df2 = spark.createDataFrame(pandas_df)
df2.show()

# 3. Создание из CSV файла (который мы сгенерировали ранее)
df3 = spark.read.csv("../../data/employees.csv", header=True, inferSchema=True)
df3.show()

+-------+---+---------+
|   name|age|      job|
+-------+---+---------+
|  Alice| 25| Engineer|
|    Bob| 30|  Manager|
|Charlie| 35|Developer|
|  Diana| 28|  Analyst|
+-------+---+---------+

+-----+---+---------------+
| name|age|            job|
+-----+---+---------------+
|  Eve| 32|       Designer|
|Frank| 27|             QA|
|Grace| 29|Product Manager|
+-----+---+---------------+

+---+-------+---+------+----------+----------+---------+
| id|   name|age|salary|department| hire_date|is_active|
+---+-------+---+------+----------+----------+---------+
|  1| User_1| 78|100957|        HR|2025-05-18|     true|
|  2| User_2| 76| 52377|        IT|2024-08-13|    false|
|  3| User_3| 43| 50963|   Finance|2023-08-05|     true|
|  4| User_4| 50|110839|   Finance|2021-01-19|     true|
|  5| User_5| 77|148058| Marketing|2023-10-11|    false|
|  6| User_6| 34| 33522|        HR|2024-12-08|     true|
|  7| User_7| 73| 47237|        HR|2025-06-18|    false|
|  8| User_8| 41| 40584|        HR|2023-

#### Изучение структуры

In [14]:
# Просмотр схемы DataFrame
df3.printSchema()

# Базовая информация о DataFrame
print(f"Кол-во строк: {df3.count()}")
print(f"Кол-во столбцов: {len(df3.columns)}")
print(f"Столбцы: {df3.columns}")

# Статистика по числовым столбцам
df3.describe().show()

# Просмотр уникальных значений
df3.select("department").distinct().show()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: integer (nullable = true)
 |-- department: string (nullable = true)
 |-- hire_date: date (nullable = true)
 |-- is_active: boolean (nullable = true)

Кол-во строк: 10000
Кол-во столбцов: 7
Столбцы: ['id', 'name', 'age', 'salary', 'department', 'hire_date', 'is_active']
+-------+------------------+---------+----------------+-----------------+----------+
|summary|                id|     name|             age|           salary|department|
+-------+------------------+---------+----------------+-----------------+----------+
|  count|             10000|    10000|           10000|            10000|     10000|
|   mean|            5000.5|     NULL|         48.8047|       89558.4295|      NULL|
| stddev|2886.8956799071675|     NULL|18.2766124149454|34605.27441500447|      NULL|
|    min|                 1|   User_1|              18|            30000|   Finance|
|    max| 

#### Простые трансформации и действия

In [17]:
# Трансформации (не выполняются сразу)
filtered_df = df3.filter(df3.age > 30)
selected_df = filtered_df.select("name", "age", "salary")

# Action (запускает выполнение)
result = selected_df.collect()
print(f"Результат {len(result)}")

# Показать результат
selected_df.show(10)

# Другие действия
print(f"Количество сотрудников старше 30: {filtered_df.count()}")
print(f"Первые 3 строки: {selected_df.head(3)}")


Результат 7891
+-------+---+------+
|   name|age|salary|
+-------+---+------+
| User_1| 78|100957|
| User_2| 76| 52377|
| User_3| 43| 50963|
| User_4| 50|110839|
| User_5| 77|148058|
| User_6| 34| 33522|
| User_7| 73| 47237|
| User_8| 41| 40584|
| User_9| 42| 62464|
|User_11| 79| 61546|
+-------+---+------+
only showing top 10 rows

Количество сотрудников старше 30: 7891
Первые 3 строки: [Row(name='User_1', age=78, salary=100957), Row(name='User_2', age=76, salary=52377), Row(name='User_3', age=43, salary=50963)]


#### Изучение плана выполнения

In [20]:
# Просмотр логического плана
filtered_df.explain()

# Проверка lazy evaluation
print("DataFrame создан, но вычисления не выполнены")
filtered_df  # Только создание объекта
print("Теперь выполним действие:")
filtered_df.count()  # Теперь вычисления происходят

== Physical Plan ==
*(1) Filter (isnotnull(age#171) AND (age#171 > 30))
+- FileScan csv [id#169,name#170,age#171,salary#172,department#173,hire_date#174,is_active#175] Batched: false, DataFilters: [isnotnull(age#171), (age#171 > 30)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/jovyan/work/data/employees.csv], PartitionFilters: [], PushedFilters: [IsNotNull(age), GreaterThan(age,30)], ReadSchema: struct<id:int,name:string,age:int,salary:int,department:string,hire_date:date,is_active:boolean>


DataFrame создан, но вычисления не выполнены
Теперь выполним действие:


7891